# CAÇANDO EC

In [ ]:
import boto3

TABLE = boto3.session.Session(region_name='us-east-1', profile_name='sofie').resource('dynamodb').Table('table_micro_task_in_person')

params = dict()

while True:
    response = TABLE.scan(**params)
    
    for each in response.get('Items') :
        print(f'{each["task_id"]} - {each["sofie_place"]["name"]}')
    
    last_key = response.get('LastEvaluatedKey')
    if not last_key:
        break
        
    params['ExclusiveStartKey'] = last_key


In [ ]:
import boto3

TABLE = boto3.session.Session(region_name='us-east-1', profile_name='sofie').resource('dynamodb').Table('table_micro_task_execution')

params = dict()

while True:
    response = TABLE.scan(**params)
    
    for each in response.get('Items'):
        execution = each['execution']
        for step in execution:
            if step['context'] == 'SYSTEM:TASK:POSTPONE_OR_CANCEL':
                if step['response'] == 'POSTPONE':
                    print(f"{each['task_id']} - {each['execution_id']} - {step['context']} - {step['response']}")
    
    last_key = response.get('LastEvaluatedKey')
    if not last_key:
        break
        
    params['ExclusiveStartKey'] = last_key

# ACURACIDADE DAS EXECUÇÕES E RESULTADO

In [ ]:
import boto3

TABLE = boto3.session.Session(region_name='us-east-1', profile_name='sofie').resource('dynamodb').Table('table_micro_task_execution')

params = dict()

while True:
    response = TABLE.scan(**params)
    
    for each in response.get('Items'):
        if 'location' in each:
            print(f"{each['task_id']} - {each['who']} - {each['location']['acc']}")
    
    last_key = response.get('LastEvaluatedKey')
    if not last_key:
        break
        
    params['LastEvaluatedKey'] = last_key

# LISTANDO ECs EXECUTADOS POR UM SOFIER

In [4]:
import boto3
import geopy.distance

WHO = 'pritybenites@gmail.com'

TABLE = boto3.session.Session(region_name='us-east-1', profile_name='sofie').resource('dynamodb').Table('table_micro_task_execution')
TABLE_TASK = boto3.session.Session(region_name='us-east-1', profile_name='sofie').resource('dynamodb').Table('table_micro_task_in_person')

params = dict()
while True:
    response = TABLE.scan(**params)
    
    for each in response.get('Items'):
        if each['task_id'] in ('7993c8f2-2232-4a5a-bac1-9e0a6cb8548a', '3e2cc519-b4dc-43b3-a822-e22935070619', 'ef144799-602e-4a1d-9dae-ebddc5f99e6d', 'f73ad59d-5c91-484b-b778-62cc2dad5303'):
            
            info_task = TABLE_TASK.get_item(Key={'task_id': each['task_id']})
            
            p1 = (info_task['Item']['sofie_place']['location']['lat'], info_task['Item']['sofie_place']['location']['lng'])
            p2 = (each['location']['lat'], each['location']['lng'])
            
            print(f"----------")
            print(f"Task ID.......: {each['task_id']}")
            print(f"Execution ID..: {each['execution_id']}")
            print(f"Data..........: {each['when']['start']}")
            print(f"Nome fantasia.: {info_task['Item']['receita_ws']['fantasia'] or info_task['Item']['receita_ws']['nome']}")
            print(f"Endereço......: {info_task['Item']['address']['formatted_address']}")
            print(f"Coordenadas...: Lat: {info_task['Item']['sofie_place']['location']['lat']} Lng: {info_task['Item']['sofie_place']['location']['lng']}")
            print(f"Executado em..: Lat: {each['location']['lat']} Lng: {each['location']['lng']} Acc: {each['location']['acc']}")
            print(f"Distância.....: {geopy.distance.distance(p1, p2).m} metros")
            print(f"----------\n")
    
    last_key = response.get('LastEvaluatedKey')
    if not last_key:
        break
        
    params['LastEvaluatedKey'] = last_key


----------
Task ID.......: 7993c8f2-2232-4a5a-bac1-9e0a6cb8548a
Execution ID..: 5d989e66-d3f5-11e9-95fe-5eef2314d224
Data..........: 2019-09-10T18:04:00
Nome fantasia.: COMERCIO DE CARNES NEVA LTDA
Endereço......: Rua Herbart 47, 47, BOX 106 - Lapa, São Paulo
Coordenadas...: Lat: -23.5193882 Lng: -46.7018484
Executado em..: Lat: -23.5192662 Lng: -46.7018919 Acc: 15.736
Distância.....: 14.223245328130993 metros
----------

----------
Task ID.......: 3e2cc519-b4dc-43b3-a822-e22935070619
Execution ID..: 6f159d8e-d3f4-11e9-95fe-5eef2314d224
Data..........: 2019-09-10T17:57:20
Nome fantasia.: VINTEM CARNES LTDA
Endereço......: Rua Herbart 47, 47, BOX 31 - Lapa, São Paulo
Coordenadas...: Lat: -23.5193882 Lng: -46.7018484
Executado em..: Lat: -23.5193827 Lng: -46.7020031 Acc: 9.32
Distância.....: 15.810644028026923 metros
----------

----------
Task ID.......: ef144799-602e-4a1d-9dae-ebddc5f99e6d
Execution ID..: dc38dab4-d3f6-11e9-95fe-5eef2314d224
Data..........: 2019-09-10T18:14:42
Nome fan